In [125]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings('ignore')

In [126]:
# Selecionando campeonato 

#https://www.betexplorer.com/soccer/brazil/serie-b-2022/results/

nation = "brazil"
league = "serie-b"
season = "2022"

url = f"https://www.betexplorer.com/soccer/{nation}/{league}-{season}/results/"
soup = BeautifulSoup(requests.get(url).content, "html.parser")

In [127]:
# Webscraping do site BetExplorer

def get_odd_or_text(td):
    if "data-odd" in td.attrs:
        return td["data-odd"]

    odd = td.select_one("[data-odd]")
    if odd:
        return odd["data-odd"]

    return td.get_text(strip=True)

all_data = []
for row in soup.select(".table-main tr:has(td)"):
    tds = [get_odd_or_text(td) for td in row.select("td")]
    round_ = row.find_previous("th").find_previous("tr").th.text
    all_data.append([round_, *tds])

In [128]:
#Organizando DataFrame

df = pd.DataFrame(all_data, columns=["Rodada", "Partida", "Placar", "Odd_H", "Odd_D", "Odd_A", "Date"])
df['League'] = league

In [129]:
#Ajustando quando o time tem uma separação -

df["Partida"].replace("Chapecoense-SC", "Chapecoense", regex=True, inplace=True)

In [130]:
df['Home'] = [i.split('-')[0] for i in df['Partida']]
df['Away'] = [i.split('-')[1] for i in df['Partida']]

In [131]:
df = df.iloc[::-1]

In [132]:
df['Goals_H'] = [i.split(':', 1)[0] for i in df['Placar']]
df['Goals_A'] = [i.split(':', 1)[1] for i in df['Placar']]
df

,Rodada,Partida,Placar,Odd_H,Odd_D,Odd_A,Date,League,Home,Away,Goals_H,Goals_A
379,1. Round,Vasco-Vila Nova FC,1:1,1.89,3.10,4.39,09.04.2022,serie-b,Vasco,Vila Nova FC,1,1
378,1. Round,Sport Recife-Sampaio Correa,1:0,1.90,3.08,4.41,09.04.2022,serie-b,Sport Recife,Sampaio Correa,1,0
377,1. Round,Ponte Preta-Gremio,0:0,4.30,3.21,1.86,09.04.2022,serie-b,Ponte Preta,Gremio,0,0
376,1. Round,Chapecoense-Ituano,1:1,2.45,2.93,3.02,09.04.2022,serie-b,Chapecoense,Ituano,1,1
375,1. Round,Brusque-Guarani,1:0,2.61,2.96,2.79,09.04.2022,serie-b,Brusque,Guarani,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
4,38. Round,Operario-Novorizontino,0:3,7.36,4.39,1.40,06.11.2022,serie-b,Operario,Novorizontino,0,3
3,38. Round,Ituano-Vasco,0:1,2.41,3.01,2.99,06.11.2022,serie-b,Ituano,Vasco,0,1
2,38. Round,Guarani-Chapecoense,1:0,2.28,3.12,3.10,06.11.2022,serie-b,Guarani,Chapecoense,1,0
1,38. Round,Cruzeiro-CSA,3:2,1.58,3.75,5.52,06.11.2022,serie-b,Cruzeiro,CSA,3,2


In [133]:
def Result(df):

    if df['Goals_H'] > df['Goals_A']:
       return "H"
    if df['Goals_H'] == df['Goals_A']:
       return "D"
    if df['Goals_H'] < df['Goals_A']:
       return "A"

In [134]:
df['Result'] = df.apply(Result,axis=1)

In [135]:
for i in range(38):
  i = i + 1
  df.replace(str(i)+". Round", i, inplace=True)

df.reset_index(inplace=True, drop=True)
df.index = df.index.set_names(['Nº'])
df = df.rename(index=lambda x: x + 1)

In [136]:
df.drop(['Partida','Placar'],axis=1,inplace=True)

In [137]:
df = df[['League','Date','Rodada','Home','Away','Odd_H','Odd_D','Odd_A','Goals_H','Goals_A','Result']]

In [138]:
df.to_excel(league+"_"+season+".xlsx")